In [4]:
%pip install nfl_data_py
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nfl_data_py as nfl
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [8]:
years = [2023]
nfl.import_weekly_data(years=years, downcast= True)
#Returns weekly data for the years and columns specified
#years : required, list of years to pull data for (earliest available is 1999)
#columns : optional, list of columns to pull data for
#downcast : converts float64 columns to float32, reducing memory usage by ~30%. Will slow down initial load speed ~50%

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NYJ,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,4,REG,...,0.0,0.483465,0,0.000000,0.031250,-0.012397,0.038197,0.0,0.8,1.8
2,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,7,REG,...,1.0,1.437224,0,3.200000,0.034483,0.094340,0.117762,0.0,1.6,2.6
3,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,11,REG,...,0.0,-0.547367,0,0.000000,0.045455,0.036885,0.094001,0.0,0.0,0.0
4,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,14,REG,...,0.0,-0.190257,0,0.000000,0.033333,-0.012987,0.040909,0.0,0.4,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5081,00-0039165,Z.Charbonnet,Zach Charbonnet,RB,RB,https://static.www.nfl.com/image/private/f_aut...,SEA,2023,13,REG,...,1.0,2.952896,0,6.500000,0.052632,0.024590,0.096160,0.0,15.9,16.9
5082,00-0039165,Z.Charbonnet,Zach Charbonnet,RB,RB,https://static.www.nfl.com/image/private/f_aut...,SEA,2023,14,REG,...,0.0,-0.787724,0,-0.666667,0.032258,-0.025532,0.030515,0.0,4.8,5.8
5083,00-0039165,Z.Charbonnet,Zach Charbonnet,RB,RB,https://static.www.nfl.com/image/private/f_aut...,SEA,2023,15,REG,...,0.0,-0.442067,0,0.000000,0.031250,0.012658,0.055736,0.0,1.6,1.6
5084,00-0039165,Z.Charbonnet,Zach Charbonnet,RB,RB,https://static.www.nfl.com/image/private/f_aut...,SEA,2023,16,REG,...,0.0,-1.961893,0,0.307692,0.055556,-0.056034,0.044109,0.0,-0.4,0.6


In [15]:
qb_columns = ['passer_player_name', 'pass_attempt', 'complete_pass', 'pass_touchdown', 'yards_gained', 'interception']
rb_columns = ['rusher_player_name', 'rush_attempt', 'rush_touchdown', 'yards_gained']
wr_columns = ['receiver_player_name', 'complete_pass', 'pass_touchdown', 'yards_gained']

qb_data = nfl.import_pbp_data(years=[2023], columns=qb_columns)
rb_data = nfl.import_pbp_data(years=[2023], columns=rb_columns)
wr_data = nfl.import_pbp_data(years=[2023], columns=wr_columns)

2023 done.
Downcasting floats.
2023 done.
Downcasting floats.
2023 done.
Downcasting floats.


In [16]:
# Calculate quarterback stats
qb_stats = qb_data.groupby('passer_player_name').agg({
    'pass_attempt': 'sum',
    'complete_pass': 'sum',
    'pass_touchdown': 'sum',
    'yards_gained': 'sum',
    'interception': 'sum'
})

# Calculate running back stats
rb_stats = rb_data.groupby('rusher_player_name').agg({
    'rush_attempt': 'sum',
    'rush_touchdown': 'sum',
    'yards_gained': 'sum'
})

# Calculate wide receiver stats
wr_stats = wr_data.groupby('receiver_player_name').agg({
    'complete_pass': 'sum',
    'pass_touchdown': 'sum',
    'yards_gained': 'sum'
})

In [17]:
# Rank quarterbacks
qb_stats['fantasy_points'] = qb_stats['complete_pass'] * 0.4 + qb_stats['pass_touchdown'] * 4 - qb_stats['interception'] * 2
top_qbs = qb_stats.sort_values(by='fantasy_points', ascending=False).head(10)

# Rank running backs
rb_stats['fantasy_points'] = rb_stats['rush_touchdown'] * 6 + rb_stats['yards_gained'] * 0.1
top_rbs = rb_stats.sort_values(by='fantasy_points', ascending=False).head(10)

# Rank wide receivers
wr_stats['fantasy_points'] = wr_stats['pass_touchdown'] * 6 + wr_stats['yards_gained'] * 0.1
top_wrs = wr_stats.sort_values(by='fantasy_points', ascending=False).head(10)

In [18]:
# Display the top 20 quarterbacks
print("Top 20 Quarterbacks:")
print(top_qbs.head(20))

# Display the top 20 running backs
print("\nTop 20 Running Backs:")
print(top_rbs.head(20))

# Display the top 20 wide receivers
print("\nTop 20 Wide Receivers:")
print(top_wrs.head(20))

Top 20 Quarterbacks:
                    pass_attempt  complete_pass  pass_touchdown  yards_gained  \
passer_player_name                                                              
D.Prescott                 595.0          379.0            32.0        3986.0   
J.Goff                     607.0          384.0            28.0        4071.0   
P.Mahomes                  624.0          401.0            27.0        3997.0   
T.Tagovailoa               564.0          371.0            28.0        4280.0   
J.Love                     578.0          345.0            30.0        3613.0   
B.Mayfield                 575.0          344.0            28.0        3701.0   
B.Purdy                    472.0          308.0            31.0        4127.0   
J.Allen                    566.0          355.0            27.0        3815.0   
D.Carr                     551.0          353.0            21.0        3417.0   
R.Wilson                   495.0          297.0            26.0        2816.0   

      